In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait # 화면이 나올때까지 대기
from selenium.webdriver.support import expected_conditions as EC    # 화면상태체크
import requests
from urllib.parse import quote
from bs4 import BeautifulSoup
import time
import os
import csv
import re
import base64
from geopy.geocoders import Nominatim
from datetime import datetime

In [ ]:
import pandas as pd

MENUS_CSV = "output1/menus.csv"   # ← 실제 파일 위치에 맞게 수정

menus_df = pd.read_csv(MENUS_CSV)
IMAGE_DIR = "food_images"
IMAGES_CSV = "images.csv"
if not os.path.exists(IMAGES_CSV):
    with open(IMAGES_CSV, "w", newline="", encoding="utf-8-sig") as f:
        writer = csv.writer(f)
        writer.writerow([
            "resno",
            "location_id",
            "locationDetail_id",
            "foodMenu_id",
            "imgno",
            "img_url",
            "img_nm",
            "date"
        ])
imgno = 1
browser = webdriver.Chrome()
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
    "Referer": "www.bluer.co.kr"
}
def make_image_name(foodMenu_id, category="pasta"):
    return f"{category}_{foodMenu_id}.jpg"

def fetch_food_image(browser, query, resno, location_id, locationDetail_id, foodMenu_id, imgno):
    search_url = f"https://search.naver.com/search.naver?ssc=tab.image.all&where=image&sm=tab_jum&query={quote(query)}"
    
    try:
        browser.get(search_url)
        time.sleep(3) # 페이지 로딩 대기
        
        soup = BeautifulSoup(browser.page_source, "lxml")
        dataId = soup.find("div",{"class":"image_tile _fe_image_tab_grid"})
        imgs = dataId.find_all("img")
        for i, img in enumerate(imgs[:2]):
            img_src = img.get('src')
            print(img_src)

            if not img_src:
                print(f"❌ 이미지 없음: {query}")
                return imgno

            img_nm = f"food_{foodMenu_id}_{imgno}.jpg"
            save_path = os.path.join(IMAGE_DIR, img_nm)

            # 이미지 저장
            response = requests.get(img_src, headers=headers, timeout=10)
            if response.status_code == 200:
                with open(save_path, "wb") as f:
                    f.write(response.content)

                # CSV 기록
                with open(IMAGES_CSV, "a", newline="", encoding="utf-8-sig") as f:
                    csv.writer(f).writerow([
                        resno, location_id, locationDetail_id, 
                        foodMenu_id, imgno, save_path, img_nm, 
                        datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                    ])
                print(f"✅ 성공: {query} -> {img_nm}")
                return imgno + 1
            
    except Exception as e:
        print(f"⚠️ {query} 시도 중 에러 발생 (사이트 구조 변경 가능성)")
        
    return imgno

# --- 실행부 ---
# IMAGE_DIR 폴더가 없으면 생성
if not os.path.exists(IMAGE_DIR):
    os.makedirs(IMAGE_DIR)

for _, row in menus_df.iterrows():
    # 검색 정확도를 위해 '음식' 키워드 추가
    search_query = f"{row['fnm']} 음식" 
    
    imgno = fetch_food_image(
        browser, search_query, 
        row["resno"], row["location_id"], row["locationDetail_id"], 
        row["fno"], imgno
    )
    time.sleep(3) # 구글 차단 방지를 위한 간격

In [ ]:
import pandas as pd

MENUS_CSV = "output1/menus.csv"   # ← 실제 파일 위치에 맞게 수정

menus_df = pd.read_csv(MENUS_CSV)
IMAGE_DIR = "food_images"
IMAGES_CSV = "images.csv"
if not os.path.exists(IMAGES_CSV):
    with open(IMAGES_CSV, "w", newline="", encoding="utf-8-sig") as f:
        writer = csv.writer(f)
        writer.writerow([
            "resno",
            "location_id",
            "locationDetail_id",
            "foodMenu_id",
            "imgno",
            "img_url",
            "img_nm",
            "date"
        ])
imgno = 503
browser = webdriver.Chrome()
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
    "Referer": "www.bluer.co.kr"
}
def make_image_name(foodMenu_id, category="pizza"):
    return f"{category}_{foodMenu_id}.jpg"

def fetch_food_images(browser, query, resno, location_id, locationDetail_id, foodMenu_id):
    global imgno  # 함수 외부의 imgno 변수를 사용하겠다고 선언
    
    search_url = f"https://search.naver.com/search.naver?ssc=tab.image.all&where=image&sm=tab_jum&query={quote(query)}"
    
    try:
        browser.get(search_url)
        time.sleep(3) 
        
        soup = BeautifulSoup(browser.page_source, "lxml")
        grid = soup.find("div", {"class": "image_tile _fe_image_tab_grid"})
        if not grid:
            print(f"❌ 검색 결과 그리드를 찾을 수 없음: {query}")
            return

        imgs = grid.find_all("img")
        
        saved_count = 0
        for i, img in enumerate(imgs):
            if saved_count >= 2: break 
            
            img_src = img.get('src')
            if not img_src or img_src.startswith('data:image'):
                continue

            # 파일명은 메뉴ID와 메뉴별 순번(1,2)을 조합하여 생성
            img_seq = saved_count + 1 
            img_nm = f"food_pizza_{foodMenu_id}_{img_seq}.jpg"
            save_path = os.path.join(IMAGE_DIR, img_nm)

            try:
                response = requests.get(img_src, headers=headers, timeout=10)
                if response.status_code == 200:
                    with open(save_path, "wb") as f:
                        f.write(response.content)

                    # CSV 기록 시 전역 변수인 imgno를 사용
                    with open(IMAGES_CSV, "a", newline="", encoding="utf-8-sig") as f:
                        csv.writer(f).writerow([
                            resno, location_id, locationDetail_id, 
                            foodMenu_id, imgno, save_path, img_nm, 
                            datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                        ])
                    
                    print(f"✅ [전체번호:{imgno}] 저장: {query} -> {img_nm}")
                    
                    saved_count += 1
                    imgno += 1  # 데이터 저장 성공 시에만 전체 일련번호 1 증가
                    
            except Exception as e:
                print(f"   - 이미지 다운로드 실패: {e}")
            
    except Exception as e:
        print(f"⚠️ {query} 시도 중 에러 발생: {e}")

# --- 실행부 ---
for _, row in menus_df.iterrows():
    search_query = f"{row['fnm']} 음식" 
    
    # 이제 imgno를 밖에서 관리하지 않고 함수 내부에서 처리
    fetch_food_images(
        browser, search_query, 
        row["resno"], row["location_id"], row["locationDetail_id"], 
        row["fno"]
    )
    time.sleep(2)

In [ ]:
browser.close()